# CONFIDENCE SCORE OF LISTINGS IN OUR DATABASE

* **Confidence Score** - This is a measure of how alive a listing is based on the properties we have in our database. 


* The **objective** of this project is to calculate and predict the **confidence score (range between 0 and 1)** of new listings posted in our database. 


* Some of the **main metrics** that can inform us of how alive a listing is are the following fields :

    1. Social Media Presence i.e. Instagram, Facebook, LinkedIn, Twitter
    2. Domain of email i.e. Yahoo, Gmail, Hotmail
    3. Type of Phone i.e. Landline, Mobile, VOIP etc
    4. Presence of website and its status i.e. active or inactive
    5. Last Website Activity i.e. Last Modified/Refreshed, Last Updated, Age of the site etc. 
    6. Data Source i.e. Yellow Pages, Scrapped profiles, Business Lists etc.
    7. Category of Listing i.e. Finance, Accomodation, Retail, Insurance, Healthcare etc
    8. Health Score i.e. fullness/completeness of a profile
    

* The **major assumptions** we are working with for this task include :
    1. Listings using a contact detail (*mobile number/email address*) have a higher probability of being alive
    2. Listings with a more complete profile(*higher health score*) have a higher probability of being alive
    3. Listings scraped from Yellow Pages are more likely to be alive than listings scraped from business lists
    4. Listings with an ISP provider e.g. Safaricom, Airtel, MTN have a higher probability of being alive
    5. Listings from Kenya, Ethiopia, South Africa, Ghana, Nigeria etc have a higher probability of being alive as a result of easy verification/reachability. 
    
    
* Some of the **biases** we are working with in this project include :
    1. Majority of the listings in our database are from countries like Ethiopia, South Africa, Kenya, Nigeria and Ghana. 
    2. Majority of the listings in our dataset are in the *General Merchants* category.
    
    
* We are going to use **unsupervised machine learning** in this project due to the fact that there is **no ground truth** as to whether a listing is alive or not when compiling them from a data source. The only way we can tell whether a listing is alive or not is by verifying using our call centre team thus the listing property *is_verified* in our database. 


* Why not use a **Supervised Machine Learning** algorithm ? This is because using the **is_verified** field as our target variable will not be accurate since majority of the listings in our database are yet to be verified in the first place (0 - 'pending verification', 1 - 'verified', 2 - 'rejected'). 


* **Suggestion** : In the near future, we can purpose to verify existing listings much faster so that we can use the **is_verified** field as a reference point (target variable) with the purpose of predicting the confidence score of new listings being added to the database. 

## Importing the necessary libraries into our environment

In [100]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier
from sklearn.cluster import KMeans
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import pairwise_distances, silhouette_score, calinski_harabasz_score

## Reading the datasets

We start by compiling all the train data sets from the database and concatenating them into one file. 

In [101]:
# Concat all the files into one (Training Files)
csv_files = [r"C:\Users\derek\Downloads\training_data(1).csv", r"C:\Users\derek\Downloads\training_data(2).csv", r"C:\Users\derek\Downloads\training_data(3).csv", r"C:\Users\derek\Downloads\training_data(4).csv", r"C:\Users\derek\Downloads\training_data(5).csv", r"C:\Users\derek\Downloads\training_data(6).csv", r"C:\Users\derek\Downloads\training_data(7).csv", r"C:\Users\derek\Downloads\training_data(8).csv", r"C:\Users\derek\Downloads\training_data(9).csv",r"C:\Users\derek\Downloads\training_data(10).csv"]

The cell below is where we will be posting the datasets containing new listings.

In [102]:
# Read the files and preview the dataset
# csv_files = []
training_data = pd.concat([pd.read_csv(file) for file in csv_files ], ignore_index=True)
print(training_data.shape)
training_data.head()

(1000000, 21)


,c.company_id,c.company_name_en,c.country,c.category_list,c.email,c.mobile,c.landline,c.isp_provider,c.geocode,c.has_contact_number,...,c.website,c.website_status,c.data_source,c.building_name,c.hours,c.confidence_indicator,c.latitude,c.longitude,c.is_headquarter,c.is_verified
0,"""ec94955a-5258-4517-bc3a-0225c68455a7""","""Cobra Knight Security""",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,"""business_list""",NaN,NaN,NaN,NaN,NaN,NaN,1
1,"""0516fad3-d072-4a9f-bb81-d23dbb8f8575""","""Medhaniyalem""","""Ethiopia""","""Clinics,Health Care/Personal Care and Social ...",NaN,"[""+251918774047""]",NaN,"[""Ethio Telecom""]",NaN,1.0,...,NaN,NaN,NaN,NaN,NaN,0.8677,NaN,NaN,0.0,0
2,"""263f2a48-6138-43ff-b459-ce95c9e72de3""","""Vertigo""","""South Africa""","""Retail Trade,Clothing-Retail""",NaN,NaN,NaN,NaN,NaN,NaN,...,"[""https://vertigoclothing.co.za""]","""inactive""",NaN,NaN,NaN,NaN,-26.111350,28.05282,0.0,0
3,"""db1abf63-96de-4f58-a3fa-865351dd063d""","""Securec Air Condition Repairs and Installations""","""South Africa""","""Air Conditioning Equipment & Systems-Repairing""",NaN,"[""+27824744471""]",NaN,"[""Vodacom""]",NaN,1.0,...,NaN,NaN,"""GMB""",NaN,"""['1-7:00:00-24:00']""",NaN,-26.097207,28.03859,NaN,0
4,"""0b639719-4b07-413e-9b46-1495ff7a058e""","""Kemila Kedir Abdo""",NaN,NaN,NaN,"[""+251906307251""]",NaN,"[""Ethio Telecom""]",NaN,1.0,...,NaN,NaN,"""Ministry of Trade Ethiopia""",NaN,NaN,NaN,NaN,NaN,NaN,0


In [103]:
# Lets create a dataset that will contain the original dataset which we will use to retrieve the company names using the index. 
final = training_data
print(final.shape)

(1000000, 21)


# Data Understanding

* Some of the columns we have in our dataset include : 
    
    1. **Company Name** - name of the listing
    2. **Country / Region / location** - location of the listing
    3. **Email Adress** - email address of the listing
    4. **Mobile / Landline** - contact details of the listing
    5. **ISP Provider** - internet service provider of the listings
    6. **Website, Website Status** - presence/name of the website as well as its status i.e. active or inactive
    7. **Category List** - category of the listing
    8. **Health Score** - completeness/fullness of a listing's profile
    9. **Has Contact Number** i.e 1 or null

# Data Cleaning

**1. First step is to deal with duplicates in our dataset. We drop them so as to maintain integrity in our data.**

In [104]:
# Check for duplicates
training_data.duplicated().sum()

42611

In [105]:
# We can view the duplicate listings before dropping them and keeping the first one.
dups = training_data[training_data.duplicated(keep=False)]
dups.sort_values(by=['c.company_name_en'], ascending=True)
dups.to_clipboard()

# Keep the first duplicate based on the company name, phone, email and country
columns = ['c.company_name_en','c.category_list','c.country','c.email','c.mobile','c.landline','c.has_contact_number'] # Location_list, geocodes, 
dups = dups.drop_duplicates(subset = columns, keep='first')
print(dups.shape)

(39623, 21)


In [106]:
# Drop the duplicated records from the original dataset
training_data = training_data.drop_duplicates(subset = columns, keep='first')
print(training_data.shape)

(880167, 21)


**2. We then deal with null values in our columns. In this case we can impute the nulls with 0 for purposes of analysis and modelling.**

In [107]:
# We can view the distribution of null values as well as data types of columns in our dataset
training_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 880167 entries, 0 to 999999
Data columns (total 21 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   c.company_id            797582 non-null  object 
 1   c.company_name_en       880163 non-null  object 
 2   c.country               505235 non-null  object 
 3   c.category_list         416985 non-null  object 
 4   c.email                 112105 non-null  object 
 5   c.mobile                429531 non-null  object 
 6   c.landline              104204 non-null  object 
 7   c.isp_provider          416382 non-null  object 
 8   c.geocode               0 non-null       float64
 9   c.has_contact_number    525774 non-null  float64
 10  c.health_score          796258 non-null  float64
 11  c.website               165274 non-null  object 
 12  c.website_status        125748 non-null  object 
 13  c.data_source           455775 non-null  object 
 14  c.building_name     

In [108]:
# We can drop the columns with the highest percentage of null values as well as unnecessary fields
training_data.drop(['c.geocode', 'c.company_id', 'c.is_headquarter'], axis=1, inplace = True)

C:\Users\derek\AppData\Local\Temp\ipykernel_7688\952569595.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data.drop(['c.geocode', 'c.company_id', 'c.is_headquarter'], axis=1, inplace = True)


# Data Exploration

Considering that most variables in our dataset are categorical, we can go ahead and display their distribution in our dataset.

In [109]:
# Verified 
print(training_data['c.is_verified'].value_counts())

0    754547
1    125584
2        36
Name: c.is_verified, dtype: int64


In [110]:
# Website Status
training_data['c.website_status'].value_counts()

"inactive"    81134
"active"      44614
Name: c.website_status, dtype: int64

Here we are attempting to find the websites last activity.

In [111]:
# import re
# training_data['c.website'] = re.sub("[^a-zA-Z0-9]","", training_data['c.website'])
# training_data['c.website']
# training_data['c.domain'] = training_data['c.website'].apply(lambda x: urllib.parse.urlparse(x).netloc)

# import requests
# from bs4 import BeautifulSoup
# import re
# url = "https://woodstockcycleworks.com"
# domain = 'youtube.com' 
# r = requests.post(url, {'domains': domain, 'submit': 'submit'})
# soup = BeautifulSoup(r.content)
# for item in soup.find_all('a', href=re.compile('website-history')):
#     print(item.text)

In [112]:
# Has contact number
training_data['c.has_contact_number'].value_counts()

1.0    525774
Name: c.has_contact_number, dtype: int64

In [113]:
# Top 20 Categories in our dataset
print(training_data['c.category_list'].nunique())
training_data['c.category_list'].value_counts().head(20)

6810


"General Merchants"                                          20462
"Agriculture, Forestry, Fishing and Hunting,Animal Feeds"     5364
"Animal Feeds,Agriculture, Forestry, Fishing and Hunting"     5326
"Business,Retail Trade"                                       4777
"Warehouses"                                                  4774
"Retail Trade,Business"                                       4717
"Schools - Primary"                                           4711
"Importers,Transportation/Auto and Warehousing"               4052
"Transportation/Auto and Warehousing,Importers"               4040
"Shopping"                                                    3536
"Schools - Primary,Educational Services"                      3465
"Accommodation"                                               3429
"Educational Services,Schools - Primary"                      3403
"School Transport"                                            3357
"Retail Trade,General Merchants"                              

In [114]:
# Top 20 Countries represented in our dataset
training_data['c.country'].value_counts().head(20)

"Nigeria"         142498
"South Africa"    139816
"Ethiopia"         52819
"Kenya"            49263
"Uganda"           18600
"Zambia"           17556
"Mozambique"       15766
"Zimbabwe"         13414
"Saudi Arabia"     10815
"Botswana"         10124
"Namibia"           9509
"Ghana"             7510
"Malawi"            5385
"Tanzania"          4572
"Lesotho"           2330
"Angola"            1687
"Swaziland"         1602
"Togo"               908
"Ivory Coast"        306
"Benin"              165
Name: c.country, dtype: int64

# Feature Engineering

**First, we create new columns that will give us additional information based on the columns in our dataset.**

In [115]:
# Has website, contacts, category, country, wroking hours, in building
training_data['c.has_website'] = np.where(training_data['c.website'].isnull(),'No','Yes')
training_data['c.has_contact_detail'] = np.where(training_data['c.email'].isnull() & training_data['c.mobile'].isnull() & training_data['c.landline'].isnull(),'No','Yes')
training_data['c.has_working_hours'] = np.where(training_data['c.hours'].isnull() | training_data['c.hours'] == '""','No','Yes')
training_data['c.in_building'] = np.where(training_data['c.building_name'].isnull(),'No','Yes')

C:\Users\derek\AppData\Local\Temp\ipykernel_7688\2221469971.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['c.has_website'] = np.where(training_data['c.website'].isnull(),'No','Yes')
C:\Users\derek\AppData\Local\Temp\ipykernel_7688\2221469971.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  training_data['c.has_contact_detail'] = np.where(training_data['c.email'].isnull() & training_data['c.mobile'].isnull() & training_data['c.landline'].isnull(),'No','Yes')
C:\Users\derek\AppData\Loca

**Another important step we can do is engineer a column that will hold the number of non-missing row values for each listing in our database. This is based on our assumption that listings with more complete profiles have a higher probability of being alive.**

In [116]:
# Calculate the number of non-missing values in each row
training_data = training_data.fillna('')
training_data['num_nonmissing'] = training_data.apply(lambda row: sum(row != ''), axis=1)

In [117]:
# Fill the empty strings with 0 after creating the num_nonmissing column
location = ['c.latitude', 'c.longitude', 'c.has_contact_number', 'c.health_score']
training_data[location] = training_data[location].replace('', 0)

# Preprocessing the dataset

**In order to perform modelling techniques, we need to encode the categorical variables while standardizing/normalizing the numerical variables.** 

Considering that our data may have outliers when it comes to latitude and longitude, we will resort to using standardization for the numerical features since this preprocessing technique is not affected by outliers in our data. 

In [118]:
# Separate categorical and numerical features
cat_cols = ['c.country', 'c.category_list', 'c.website_status', 'c.isp_provider', 'c.data_source', 'c.has_working_hours', 'c.in_building', 'c.has_website', 'c.has_contact_detail']
num_cols = ['c.latitude', 'c.longitude', 'c.health_score']

# Label encode categorical features
for col in cat_cols:
    training_data[col] = training_data[col].astype('category')
    training_data[col] = training_data[col].cat.codes

# Standardize numerical features
scaler = StandardScaler()
training_data[num_cols] = scaler.fit_transform(training_data[num_cols])

In [119]:
# Before fitting our Kmeans cluster algorithm, we can drop the unnecessary variables in our dataset
training_data.drop(['c.company_name_en','c.email','c.mobile','c.landline','c.website','c.building_name','c.hours','c.confidence_indicator'], axis=1, inplace=True)

In [120]:
# Preview of the data after encoding and dropping unnecessary features.
training_data.head()

,c.country,c.category_list,c.isp_provider,c.has_contact_number,c.health_score,c.website_status,c.data_source,c.latitude,c.longitude,c.is_verified,c.has_website,c.has_contact_detail,c.has_working_hours,c.in_building,num_nonmissing
0,0,0,0,0.0,-0.031193,0,109,0.196873,-0.539684,1,0,0,0,0,8
1,13,1136,27,1.0,-0.031193,0,0,0.196873,-0.539684,0,0,1,0,0,13
2,40,5511,0,0.0,-0.031193,2,0,-2.097785,1.552553,0,1,0,0,0,13
3,40,293,114,1.0,0.986903,0,45,-2.096542,1.551491,0,0,1,0,0,16
4,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11


Considering the distribution of data is uneven in our dataset, we will randomize our train dataset before splitting into a 70-30 split for modelling purposes.

In [121]:
# Randomize the dataset before splitting into train and test
trains = training_data.sample(frac=1)
print(trains.shape)
trains.head()

(880167, 15)


,c.country,c.category_list,c.isp_provider,c.has_contact_number,c.health_score,c.website_status,c.data_source,c.latitude,c.longitude,c.is_verified,c.has_website,c.has_contact_detail,c.has_working_hours,c.in_building,num_nonmissing
189874,0,0,27,1.0,-0.031193,0,38,0.196873,-0.539684,0,0,1,0,0,11
579881,30,0,0,0.0,-0.031193,0,0,-1.973290,1.878943,0,0,0,0,0,12
191,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11
610842,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11
87059,0,0,27,1.0,-0.031193,0,38,0.196873,-0.539684,0,0,1,0,0,11


In [122]:
# Train-Test Split (70:30 split)
trains_data = trains.iloc[0:616001] # Train set
test_data = trains.iloc[616001:880168] # Test set

## Initialize the model

Considering we want the confidence score to range between 0 and 1, we can initialize the Kmeans algorithm with 10 clusters then we'll later optimize with the best parameters using GridSearchCV. 

In [123]:
# Perform clustering on the train set data
kmeans = KMeans(n_clusters=3, random_state=42, max_iter=100)
kmeans.fit(trains_data)

KMeans(max_iter=100, n_clusters=3, random_state=42)

* In this stage, we will get the cluster labels for each data point before calculating the confidence score. 

* We will then calculate the confidence score as a fraction of the non-missing values in each cluster and assign the value to each listing in the cluster. 

In [124]:
# Get the cluster labels for each data point
cluster_labels = kmeans.predict(trains_data)

# Calculate the confidence score as the fraction of non-missing values in each cluster
num_nonmissing_means = []
for cluster in range(kmeans.n_clusters):
    mask = (cluster_labels == cluster)
    cluster_df = trains_data[mask]
    num_nonmissing_mean = cluster_df['num_nonmissing'].mean()
    num_nonmissing_means.append(num_nonmissing_mean)

confidence_scores = []
for label in cluster_labels:
    confidence_scores.append(num_nonmissing_means[label] / len(trains_data.columns))

print(confidence_scores)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [125]:
# Create a confidence score column that will hold the scores for each listing in our dataset
trains_data['confidence_score'] = confidence_scores
print(trains_data['confidence_score'].unique())
print()
trains_data.head()

[0.76852253 0.91913835 0.89198431]



C:\Users\derek\AppData\Local\Temp\ipykernel_7688\1892472745.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trains_data['confidence_score'] = confidence_scores


,c.country,c.category_list,c.isp_provider,c.has_contact_number,c.health_score,c.website_status,c.data_source,c.latitude,c.longitude,c.is_verified,c.has_website,c.has_contact_detail,c.has_working_hours,c.in_building,num_nonmissing,confidence_score
189874,0,0,27,1.0,-0.031193,0,38,0.196873,-0.539684,0,0,1,0,0,11,0.768523
579881,30,0,0,0.0,-0.031193,0,0,-1.973290,1.878943,0,0,0,0,0,12,0.768523
191,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11,0.768523
610842,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11,0.768523
87059,0,0,27,1.0,-0.031193,0,38,0.196873,-0.539684,0,0,1,0,0,11,0.768523


## Hyperparameter Tuning

**For hyperparameter tuning, we're basically searching for the best parameters that will give us the best distinction between clusters. In this case we're going to use GridSearchCV.**

In [127]:
# Import required libraries
from sklearn.model_selection import GridSearchCV

Define hyperparameter grid
param_grid = {
    'n_clusters': [3, 5, 7, 10, 15, 20],
    'init': ['k-means++', 'random'],
    'max_iter': [100, 300, 500],
    'tol': [1e-4, 1e-5, 1e-6]
}

# Create KMeans model
kmeans = KMeans()

# Perform grid search
grid_search = GridSearchCV(kmeans, param_grid, scoring='roc_auc', n_jobs=-1)
grid_search.fit(trains_data)

# Print best parameters
print('Best parameters:', grid_search.best_params_)

After doing some hyperparameter tuning, it was determined that 3 is the optimum number of clusters that we should use for our data. This will be implemented on our test data and evaluated using the silhouette score.

**Predicting the confidence score of the test dataset.**

In [128]:
# Predicting on a test set
# test_data[num_cols] = scaler.transform(test_data[num_cols])
predicted_cluster_labels = kmeans.predict(test_data)

# Calculate the confidence score as a fraction of non-missing values in each cluster
num_nonmissing_means = []
for cluster in range(kmeans.n_clusters):
    mask = (predicted_cluster_labels == cluster)
    cluster_df = test_data[mask]
    num_nonmissing_mean = cluster_df['num_nonmissing'].mean()
    num_nonmissing_means.append(num_nonmissing_mean)

test_confidence_scores = []
for label in predicted_cluster_labels:
    test_confidence_scores.append(num_nonmissing_means[label] / len(test_data.columns))

print(test_confidence_scores)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [129]:
# Create a confidence score column
test_data['confidence_score'] = test_confidence_scores
print(test_data['confidence_score'].unique())
test_data.head()

[0.76921841 0.91984279 0.89174415]


C:\Users\derek\AppData\Local\Temp\ipykernel_7688\2115312431.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data['confidence_score'] = test_confidence_scores


,c.country,c.category_list,c.isp_provider,c.has_contact_number,c.health_score,c.website_status,c.data_source,c.latitude,c.longitude,c.is_verified,c.has_website,c.has_contact_detail,c.has_working_hours,c.in_building,num_nonmissing,confidence_score
813292,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11,0.769218
339324,33,0,54,1.0,0.986903,0,0,0.769772,-0.286691,0,0,1,0,0,15,0.769218
199111,22,2934,77,1.0,0.986903,1,45,0.196873,-0.539684,0,1,1,0,0,15,0.919843
641462,33,0,0,0.0,-1.049289,0,0,0.196873,-0.539684,0,0,0,0,0,10,0.769218
17357,0,0,27,1.0,-0.031193,0,53,0.196873,-0.539684,0,0,1,0,0,11,0.769218


# Model Evaluation

**We'll be using the silhouette score to evaluate the consistency within clusters of data.**

In [130]:
# Compute the silhouette score
from sklearn.metrics import silhouette_score
silhouette = silhouette_score(test_data, predicted_cluster_labels)

print("Silhouette score: ", silhouette)

Silhouette score:  0.7973394193735287


# Observations

1. When using **10 clusters** in our Kmeans cluster algorithm, our **silhouette score was 0.74** which is pretty accurate. 


2. When using **3 clusters**, the **silhouette score improved to 0.8** which is much more accurate than when using 10 clusters. 


3. For new listings, we will use 3 clusters as a parameter in our kMeans algorithm so as to get the best distinction between listings in our database. 